In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from xgboost import XGBClassifier

from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re, string, unicodedata
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import pandas as pd
import numpy as np
import re,string
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
import gensim.downloader as api
from nltk.tokenize import word_tokenize,sent_tokenize


In [ ]:
!pip install sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/IMDB/IMDB Dataset.csv",header=0)
data

In [ ]:
data.isna().sum()

In [ ]:
for i in range(data.shape[1]):
  print(data.iloc[:,i].unique())

In [ ]:
for i in range(data.shape[1]):
  print(data.iloc[:,i].value_counts())

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
data['sentiment'].value_counts()

In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
data['review']=data['review'].apply(denoise_text)

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
data['review']=data['review'].apply(remove_special_characters)

In [ ]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
data['review']=data['review'].apply(simple_stemmer)

In [ ]:
data

In [ ]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
data['review']=data['review'].apply(remove_stopwords)

In [ ]:
data.sentiment = [ 1 if each == "positive" else 0 for each in data.sentiment]

In [ ]:
data

In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)

In [ ]:
# Convert text to vectors using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_train = vectorizer.fit_transform(X_train)
tfidf_test = vectorizer.transform(X_test)

In [ ]:
# Create a logistic regression object
lr = LogisticRegression()

In [ ]:
# Fit the model to the training data
lr.fit(tfidf_train, y_train)

In [ ]:
# Predict the sentiment labels for the testing data
y_pred = lr.predict(tfidf_test)

In [ ]:
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)

In [ ]:
print('Test accuracy:', acc)

In [ ]:
# Train and download Word2Vec vectors
sentences = [review.split() for review in data['review']]
model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)
model.save('word2vec.model')

In [ ]:
# Convert each word to vector and represent the sentence in vector form using the word embeddings
def sentence_vector(sentence,model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv.vocab]
    if len(word_vectors) == 0:
        return np.zeros((100,))
    return np.mean(word_vectors, axis=0)

In [ ]:
# Convert each sentence in the dataset to a vector using the Word2Vec model
word2vec_train = np.array([sentence_vector(sentence, model) for sentence in data['review']])
word2vec_test = np.array(data['sentiment'])

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(word2vec_train, word2vec_test, test_size=0.2, random_state=42)

In [ ]:
# Create a logistic regression object
lr2 = LogisticRegression()

In [ ]:
# Fit the model to the training data
lr2.fit(X_train, y_train)

In [ ]:
# Predict the sentiment labels for the testing data
y_pred = lr2.predict(X_test)

In [ ]:
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)

In [ ]:
print('Test accuracy:', acc)

In [ ]:
def combine_vectors(doc):
    tfidf_vec = vectorizer.transform([doc])
    w2v_vec = sentence_vector(doc, model)
    combined_vec = np.concatenate([np.squeeze(tfidf_vec.toarray()), w2v_vec])
    return combined_vec

In [ ]:
combined_train = np.array([combine_vectors(doc) for doc in data['review']])
combined_test = np.array(data['sentiment'])

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_train, combined_test, test_size=0.2, random_state=56)

In [ ]:
lr3 = LogisticRegression()
lr3.fit(X_train, y_train)

In [ ]:
y_predlr = lr3.predict(X_test)
acc = accuracy_score(y_test, y_predlr)
print('Accuracy:', acc)

print("\n\n........................CLASSIFICATION MATRIX.....................")
print(classification_report(y_predlr,y_test))

cm=confusion_matrix(y_test, y_predlr, labels=lr3.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=lr3.classes_)
disp.plot()
plt.show()

In [ ]:
svc2= LinearSVC(penalty='l2',loss='hinge')
svc2.fit(X_train, y_train)

In [ ]:
svc1pred= svc2.predict(X_test)
svcscore = accuracy_score(y_test, svc1pred)
print("SVC Accuracy: ", round((100*svcscore),2),"%")

print("\n\n........................CLASSIFICATION MATRIX.....................")
print(classification_report(svc1pred,y_test))

cm=confusion_matrix(y_test, svc1pred, labels=svc2.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=svc2.classes_)
disp.plot()
plt.show()

In [ ]:
xgbc=  XGBClassifier()
xgbc.fit(X_train, y_train)

In [ ]:
xgb1pred= xgbc.predict(X_test)
xgbscore = accuracy_score(y_test, xgb1pred)
print("XGB Accuracy: ", round((100*xgbscore),2),"%")

print("\n\n........................CLASSIFICATION MATRIX.....................")
print(classification_report(xgb1pred,y_test))

cm=confusion_matrix(y_test, xgb1pred, labels=xgbc.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=xgbc.classes_)
disp.plot()
plt.show()

In [ ]:
!cat /proc/meminfo